In [6]:
import requests
from lxml import etree
import os
import time

if __name__ == '__main__':
    # 创建文件夹
    if not os.path.exists('./maogai/Total/'):
        os.mkdir('./maogai/Total/')

    session = requests.Session()
    detail_url = 'https://wlkc.ouc.edu.cn'
    # 使用现成的cookie直接绕过登录页面
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
        'Cookie': 'JSESSIONID=5913C61E01092FBC3AFCDF3919FCEA07; LOGIN=3139303230303331303330; UM_distinctid=17bcb2f4e579ab-0e328a4e032f8-a7d193d-144000-17bcb2f4e58315; SCREEN_NAME=5a7a5555444a5247484f6d2f7361764541302b7231673d3d; session_id=6BA05B378A8C9369E72001F2CE0D481D; s_session_id=121E1BA42BA76D38DC9AF11FEB839995; web_client_cache_guid=6f191ada-554a-459b-8898-0fe0ecd0c93c',
    }

    # 自动请求开始测试和提交 times_att为次数 每次请求间隔3s
    times_att = 0
    print('attempt started...')
    for i in range(0,times_att):
        # 请求测试
        attempt_url = 'https://wlkc.ouc.edu.cn/webapps/assessment/take/launch-redirect.jsp?new_attempt=1&course_assessment_id=_30547_1&course_id=_13492_1&content_id=_641342_1&step='           
        atmp = session.get(url=attempt_url, headers=headers)
        
        # 拿到本次测试id
        do_url = 'https://wlkc.ouc.edu.cn/webapps/assessment/take/launch.jsp?course_assessment_id=_30547_1&course_id=_13492_1&new_attempt=1&content_id=_641342_1&step='
        do = session.get(url=do_url, headers=headers)
        tree_att = etree.HTML(do.text)
        atnum = tree_att.xpath('//div[@class="takeQuestionDiv "]/@id')[0]
        print('attemptid = ' + str(atnum))

        # 提交本次测试
        submit_url = 'https://wlkc.ouc.edu.cn/webapps/assessment/do/take/saveAttempt?saveSequence=1&takePageId=1639991818110&course_assessment_id=_30547_1&course_id=_13492_1&content_id=_641342_1'
        data = {
            'blackboard.platform.security.NonceUtil.nonce.ajax': '44c94f34-2eac-4778-a8d9-97bfbde4e673',
            'data-submitted': 'true',
            'course_assessment_id': '_30547_1',
            'course_id': '_13492_1',
            'content_id': '_641342_1',
            'step': '',
            'original_user_id': '_47406_1',
            'save_and_submit': 'true',
            'timer_completion': '',
            'fileUploadType': '',
            'toggle_state': 'qShow',
            'current_question': '1',
            'current_attempt_item_id': atnum,
            'current_attempt_item_id_backup': atnum,
            'method': 'notajax',
            'saveonequestion': '',
        }
        submit = session.post(url=submit_url, headers=headers, data=data)
        print('post finished! times: '+ str(i+1))
        time.sleep(3)
    print('attempt over!')
    print('--------------------')

    # 总尝试页面
    total_url = 'https://wlkc.ouc.edu.cn/webapps/gradebook/do/student/viewAttempts?method=list&course_id=_13492_1&outcome_definition_id=_96486_1&outcome_id=_2149299_1'
    page_text_tot = session.get(url=total_url, headers=headers)
    tree_tot = etree.HTML(page_text_tot.text)
    attempt_list = tree_tot.xpath('//div[@class=" columnStep clearfix"]//a/@href')

    # 为了去重使用字典保存键值对
    danxuan = {}
    duoxuan = {}
    panduan = {}
    cnt1 = 0
    cnt2 = 0
    cnt3 = 0

    fp1 = open('./maogai/Total/单选.txt', 'w', encoding='utf-8')
    fp2 = open('./maogai/Total/多选.txt', 'w', encoding='utf-8')
    fp3 = open('./maogai/Total/判断.txt', 'w', encoding='utf-8')

    times = 0
    print('download started...')
    for attempt in attempt_list:
        time.sleep(0.5)
        times = times + 1
        new_url = detail_url + attempt
        # print(new_url)
        page_text = session.get(url=new_url, headers=headers)
        tree = etree.HTML(page_text.text)
        pro_list = tree.xpath('//ul[@id="content_listContainer"]/li')

        cnt = 0
        # 每次尝试的内容进行爬取
        for li in pro_list:
            cnt = cnt + 1
            pro_name = li.xpath('normalize-space(.//div[@class="vtbegenerated inlineVtbegenerated"])')
            pro_answer_1 = li.xpath('.//div[@class="vtbegenerated inlineVtbegenerated"]//label/text()')
            pro_answer_1 = str(pro_answer_1)
            pro_answer_2 = li.xpath('normalize-space(.//span[@class="answerTextSpan"])')

            if cnt <= 5:
                if pro_name not in danxuan:
                    cnt1 = cnt1 + 1
                    danxuan[pro_name] = pro_answer_1
                    # fp1.write(str(cnt1) + '. ' + pro_name+'\n'+pro_answer_1+'\n')
            elif cnt <= 10:
                if pro_name not in duoxuan:
                    cnt2 = cnt2 + 1
                    duoxuan[pro_name] = pro_answer_1
                    # fp2.write(str(cnt2) + '. ' + pro_name + '\n' +pro_answer_1+'\n')
            else:
                if pro_name not in panduan:
                    cnt3 = cnt3 +1
                    panduan[pro_name] = pro_answer_2
                    # fp3.write(str(cnt3) + '. ' + pro_name + '\n'+pro_answer_2+'\n')
        print('times: ' + str(times) + ' 单选: ' + str(cnt1) + ',多选：' + str(cnt2) + ',判断: ' + str(cnt3) )

    # 将字典进行按key排序实现不同章节分类进行存储
    num = 0
    for pro in sorted(danxuan):
        num = num + 1
        fp1.write(str(num)+'. '+ pro + '\n' + danxuan[pro] + '\n')
    num = 0
    for pro in sorted(duoxuan):
        num = num + 1
        fp2.write(str(num)+'. ' + pro + '\n' + duoxuan[pro] + '\n')
    num = 0
    for pro in sorted(panduan):
        num = num + 1
        fp3.write(str(num)+'. ' + pro + '\n' + panduan[pro] + '\n')

    fp1.close()
    fp2.close()
    fp3.close()
    print('total: ' + str(cnt1+cnt2+cnt3) + ' problems have been downloaded.')


attempt started...
attemptid = _17617943_1
post finished! times: 1
attemptid = _17617960_1
post finished! times: 2
attemptid = _17617994_1
post finished! times: 3
attempt over!
--------------------
download started...
times: 1 单选: 5,多选：5,判断: 5
times: 2 单选: 10,多选：10,判断: 10
times: 3 单选: 15,多选：14,判断: 14
times: 4 单选: 20,多选：19,判断: 19
times: 5 单选: 24,多选：23,判断: 24
times: 6 单选: 28,多选：27,判断: 27
times: 7 单选: 32,多选：31,判断: 31
times: 8 单选: 34,多选：35,判断: 35
times: 9 单选: 39,多选：40,判断: 39
times: 10 单选: 42,多选：43,判断: 43
times: 11 单选: 44,多选：47,判断: 46
times: 12 单选: 48,多选：51,判断: 47
times: 13 单选: 51,多选：53,判断: 49
times: 14 单选: 52,多选：56,判断: 53
times: 15 单选: 55,多选：61,判断: 58
times: 16 单选: 59,多选：63,判断: 60
times: 17 单选: 61,多选：65,判断: 61
times: 18 单选: 64,多选：68,判断: 65
times: 19 单选: 66,多选：70,判断: 68
times: 20 单选: 67,多选：72,判断: 73
times: 21 单选: 70,多选：73,判断: 76
times: 22 单选: 72,多选：74,判断: 76
times: 23 单选: 73,多选：75,判断: 79
times: 24 单选: 77,多选：78,判断: 79
times: 25 单选: 81,多选：80,判断: 83
times: 26 单选: 83,多选：84,判断: 84
times: 27 单选: 